# Código Scraping TFM 2025

A continuación se muestra el código python utilizado para la recolección de datos de coches en venta en internet. La página web de la que serán extraídos los datos es https://www.autoscout24.com/lst?atype=C&cy=D&damaged_listing=exclude&desc=0&offer=N&powertype=kw&search_id=jx8vlorwwp&sort=standard&source=homepage_categories&ustate=N%2CU.

### 1. Abrimos la página web, rechazamos cookies y efectuamos búsqueda de coches en venta

In [43]:
#Importamos todas las librerias que necesitamos 

import pandas as pd
import re
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
 
driver = webdriver.Firefox()
driver.get("https://www.autoscout24.com/lst?atype=C&cy=D&damaged_listing=exclude&desc=0&fuel=C%2CH%2CL%2CM%2CO&ocs_listing=include&offer=U&powertype=kw&search_id=19rgg7gyfkq&sort=standard&source=listpage_pagination&ustate=N%2CU")
# coches_db36_german_used_other-fuels
time.sleep(3)
driver.maximize_window()

In [44]:
# Rechazamos cookies
cookies = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[3]/button[1]")
# /html/body/div[3]/div/div/div[3]/button[1]
# /html/body/div[2]/div/div/div[3]/button[1]
cookies.click()
time.sleep(2)
cookies = driver.find_element(By.XPATH,"/html/body/div[2]/div/article/section[1]/button[2]")
cookies.click()
time.sleep(1)

In [45]:
# Creamos un diccionario y un dataframe vacíos
car_data = {}
df = pd.DataFrame([car_data])

### 2. Creamos el bucle definitivo

In [47]:
for i in range(19):
    # 1) Collect all listing elements on this page
    car_elements = driver.find_elements(
        By.XPATH,
        '//div[contains(@class, "ListItem_header")]'
    )

    for element in car_elements:
        try:
            # --- Extract make, model, and link ---
            names = element.find_elements(By.TAG_NAME, "span")
            car_make  = names[0].text.strip()
            car_model = names[1].text.strip()
            car_link  = element.find_element(By.TAG_NAME, 'a') \
                               .get_attribute('href')
            print(f"Visiting: {car_make} {car_model} at {car_link}")

            # --- Open detail in new tab ---
            driver.execute_script("window.open(arguments[0], '_blank');", car_link)
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) == 2)
            driver.switch_to.window(driver.window_handles[1])

            # --- Expand equipment if needed ---
            try:
                btn = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR,
                         "button.ExpandableDetailsSection_expandButton__Jir5n")
                    )
                )
                if btn.get_attribute("aria-expanded") == "false":
                    btn.click()
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR,
                             "dl.DataGrid_defaultDlStyle__xlLi_")
                        )
                    )
            except Exception as e:
                print("Expand button or dl elements not found:", e)

            # --- Click “See more” if present ---
            try:
                see_more = driver.find_element(
                    By.CSS_SELECTOR,
                    "button[aria-label='See more']"
                )
                see_more.click()
            except NoSuchElementException:
                print("No 'See more' button, continuing...")

            time.sleep(2)

            # --- Clean up price superscript and read price ---
            driver.execute_script("""
                const span = document.querySelector('span.PriceInfo_price__XU0aF');
                const sup  = span?.querySelector('sup');
                if (sup) sup.remove();
            """)
            car_price = driver.find_element(
                By.CSS_SELECTOR,
                "span.PriceInfo_price__XU0aF"
            ).text.strip()

            # --- Build car_data dict ---
            car_data = {
                "car_make":  car_make,
                "car_model": car_model,
                "price":     car_price
            }

            # --- Scrape all dl blocks for specs ---
            specs = driver.find_elements(
                By.CSS_SELECTOR,
                "dl.DataGrid_defaultDlStyle__xlLi_"
            )
            for dl in specs:
                dts = dl.find_elements(By.TAG_NAME, "dt")
                dds = dl.find_elements(By.TAG_NAME, "dd")
                for dt, dd in zip(dts, dds):
                    car_data[dt.text.strip()] = dd.text.strip()

            # --- Dealer info ---
            dealer = driver.find_element(
                By.CSS_SELECTOR,
                "div.TieredPricingRatingsSection_nameContainer__fMSj2"
            ).text.strip()
            car_data["dealer"] = dealer

            # Ratings number
            ratings_container = driver.find_element(
                By.CSS_SELECTOR,
                "div.DealerRatings_ratingsAndRecommendationText__c8J3h"
            )
            try:
                link = ratings_container.find_element(By.TAG_NAME, "a")
                text = link.find_element(By.TAG_NAME, "span").text
            except NoSuchElementException:
                text = ratings_container.find_element(By.TAG_NAME, "span").text
            car_data["ratings_number"] = int(text.split()[0])

            # Location
            loc_elem = driver.find_element(
                By.CSS_SELECTOR,
                "a.LocationWithPin_locationItem__tK1m5"
            )
            car_data["car_location"] = loc_elem.text.strip()

            # --- Append to DataFrame ---
            df = pd.concat([df, pd.DataFrame([car_data])], ignore_index=True)

            # --- Close tab and return to main ---
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except (StaleElementReferenceException, NoSuchElementException, TimeoutException) as e:
            print("Error on listing, skipping:", e)
            # Ensure we close any extra tab
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            continue

    # --- After finishing all elements on this page, click Next ---
    try:
        next_page = driver.find_element(
            By.CSS_SELECTOR,
            "button[aria-label='Go to next page']"
        )
        next_page.click()
        time.sleep(3)
    except NoSuchElementException:
        print("No Next button—scraping complete or last page reached.")
        break

Visiting: SEAT Arona at https://www.autoscout24.com/offers/seat-arona-1-0-tgi-fr-cng-ac-a-led-kamera-pdc-17-carplay-cng-red-d7f25d8e-2a22-41f1-bfb8-4e8385572cc3
Visiting: Skoda Superb at https://www.autoscout24.com/offers/skoda-superb-lim-1-4-tsi-iv-l-k-dsg-matrix-ahk-sthz-laurin-kl-others-brown-8f67b9dc-4ab9-4289-81b9-f70e2e0037eb
Visiting: Toyota Mirai at https://www.autoscout24.com/offers/toyota-mirai-advanced-wasserstoff-pano-hud-jbl-hydrogen-black-a18b297a-79c2-456d-ab63-90e8000a365f
Visiting: SEAT Arona at https://www.autoscout24.com/offers/seat-arona-arona-1-0-tgi-fr-cng-fast-lane-navi-led-rfk-acc-cng-white-f712d778-b177-4a6f-bc4f-72c16842f5c6
Visiting: Opel Zafira Tourer at https://www.autoscout24.com/offers/opel-zafira-tourer-c-1-6-innovation-cng-erdgas-7sitze-cng-black-82d0fe76-4bb9-49f8-bfe9-6f774412b2a9
Visiting: Volkswagen Caddy at https://www.autoscout24.com/offers/volkswagen-caddy-trendline-1-4-dsg-tgi-cgn-klima-bt-navi-pdc-others-silver-f2f30c19-a699-46ec-b48a-12c58e3ee

Visiting: Hyundai NEXO at https://www.autoscout24.com/offers/hyundai-nexo-prime-hydrogen-blue-1372f5eb-7045-4703-b900-77586bbaee3c
Visiting: Audi SQ8 at https://www.autoscout24.com/offers/audi-sq8-4-0-tfsi-quattro-ahk-hd-martix-led-standhzg-pano-others-black-bd151f08-7723-4e6c-b228-00b1093761fe
Visiting: Dacia Jogger at https://www.autoscout24.com/offers/dacia-jogger-expression-tce-100-eco-g-lpg-black-a32394ae-1793-4083-abc7-b2b5afab3c6a
Visiting: Hyundai NEXO at https://www.autoscout24.com/offers/hyundai-nexo-parkass-fla-kam-4xshz-spurh-hydrogen-silver-3b5e6538-2d12-416e-a6d8-b76c8bf7607f
Visiting: Lada Taiga at https://www.autoscout24.com/offers/lada-taiga-urban-plus-4x4-gasumbau-inkl-lpg-green-83d65d2b-c581-444f-aad0-9ad44c82f7b9
Visiting: Dacia Sandero at https://www.autoscout24.com/offers/dacia-sandero-iii-stepway-tce-100-lpg-expression-led-fahrerprofi-lpg-white-9380cf16-4cfd-41db-8357-7f0e3ac89414
Visiting: Dacia Sandero at https://www.autoscout24.com/offers/dacia-sandero-iii-ste

Visiting: Mercedes-Benz B 200 at https://www.autoscout24.com/offers/mercedes-benz-b-200-klasse-c-ngd-automatik-nav-xenon-ahk-kamera-shz-a-cng-black-65c0cb36-0342-4ade-b444-57abeef4461d
Visiting: Skoda Kamiq at https://www.autoscout24.com/offers/skoda-kamiq-1-0-tsi-dsg-monte-carlo-ahk-kamera-gasdach-others-black-c7559735-576d-4e67-85b1-3fcef0ae59c0
Visiting: Dacia Duster at https://www.autoscout24.com/offers/dacia-duster-ii-comfort-100-lpg-shz-pdc-klima-bt-zv-lpg-orange-bbf62b16-0b40-49c4-a927-5576d5fa8b02
Visiting: SEAT Arona at https://www.autoscout24.com/offers/seat-arona-1-0-tgi-xperience-vision-nav-led-shz-cng-grey-3e15f2e9-7374-45b8-ae6c-9a4929079124
Visiting: Audi A5 at https://www.autoscout24.com/offers/audi-a5-40-tfsi-g-tron-s-tronic-s-line-mat-cng-blue-26f5164c-12a8-43ca-8bdd-4c648e44f215
Visiting: Dacia Sandero at https://www.autoscout24.com/offers/dacia-sandero-tce-100-eco-g-lpg-klima-facelift-lpg-white-69fea8a5-fd21-48d2-9cd9-e40af6fd0bc3
Visiting: Skoda Kamiq at https://ww

Error on listing, skipping: Message: Unable to locate element: div.TieredPricingRatingsSection_nameContainer__fMSj2; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:552:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Visiting: Dacia Duster at https://www.autoscout24.com/offers/dacia-duster-lpg-extreme-navi-cam-klimaautomatik-others-black-2a76743e-3a52-423f-9f52-d52433c2c03c
Visiting: Dacia Sandero at https://www.autoscout24.com/offers/dacia-sandero-iii-stepway-lpg-17-zoll-allwetternavi-1-hand-lpg-silver-7e2beb54-e1aa-4648-a002-878821ce119b
Visiting: SEAT Leon at https://www.autoscout24.com/offers/seat-leon-1-5-tgi-fr-pano-navi-acc-led-r-cam-shz-c

Visiting: Chevrolet Nubira at https://www.autoscout24.com/offers/chevrolet-nubira-wagon-sx-mit-lpg-gasanlage-lpg-blue-8bfc5ac0-34c3-45c5-82e6-3c96e9af3715
Visiting: SEAT Arona at https://www.autoscout24.com/offers/seat-arona-1-0-tgi-reference-cng-klima-5-100km-cng-white-5467ad0e-bc21-43eb-b348-52e1525ed22c
Visiting: Hyundai i30 at https://www.autoscout24.com/offers/hyundai-i30-classic-lpg-blue-7950d783-1a83-402f-ac76-6326a266ddce
Visiting: Ford C-Max at https://www.autoscout24.com/offers/ford-c-max-2-0-futura-klima-kein-tuv-nr39-cng-red-dd32b09e-3b02-40d2-8f85-3fb514befceb
Visiting: Opel Combo at https://www.autoscout24.com/offers/opel-combo-c-edition-1-6-klima-5-sitze-erdgas-cng-cng-silver-f3e83a30-f46b-495b-9600-038e7b50bf5c
Visiting: SEAT Ibiza at https://www.autoscout24.com/offers/seat-ibiza-1-0-tgi-s-cng-white-99eb7bab-9bd2-4b5f-a589-4fb6b40eda00
Error on listing, skipping: Message: Unable to locate element: div.DealerRatings_ratingsAndRecommendationText__c8J3h; For documentation 

Visiting: Volkswagen Caddy at https://www.autoscout24.com/offers/volkswagen-caddy-life-familien-ecofuel-2-0-7-sitzer-klima-cng-silver-41070ba9-0ca9-4466-b746-614a305c04da
Visiting: SEAT Arona at https://www.autoscout24.com/offers/seat-arona-fr-acc-led-dab-cng-white-eb428d6a-9665-4c10-b469-20022e298968
Visiting: Suzuki SX4 at https://www.autoscout24.com/offers/suzuki-sx4-1-6-vvt-comfort-lpg-klima-shz-keyless-tuev-neu-lpg-blue-fb8c5b9e-b823-4d72-97a4-02d85212b3d8
Visiting: Volkswagen T6 Kombi at https://www.autoscout24.com/offers/volkswagen-t6-kombi-2-0-tdi-9-sitzer-klima-navi-akh-winterraeder-others-grey-0068eb7a-207d-4203-8582-6a71a9c974bb
Visiting: Volkswagen Caddy at https://www.autoscout24.com/offers/volkswagen-caddy-life-2-0-ecofuel-hu-neu-klima-cng-silver-f4ba87a9-f501-4f34-816f-c5af398192a1
Visiting: Kia Sportage at https://www.autoscout24.com/offers/kia-sportage-2-0-attract-lpg-gas-klima-ahk-tuev-neu-lpg-grey-3d4c42e4-e25a-4d15-ac0d-b3cf16d575d4
Visiting: Hyundai i30 at https://

Visiting: Dacia Sandero at https://www.autoscout24.com/offers/dacia-sandero-iii-stepway-navi-pdc-klima-dab-tempo-led-lpg-red-34ec43dd-05a8-46b5-a5e3-25c9596bed72
Visiting: Suzuki Jimny at https://www.autoscout24.com/offers/suzuki-jimny-1-3-cabrio-daktari-limited-edition-150-others-black-1dd09d8c-e822-4638-bbfe-ac400eb1c84b
Visiting: Volkswagen Polo at https://www.autoscout24.com/offers/volkswagen-polo-1-0-tgi-comfortline-navi-klima-bluetooth-cng-blue-b1c0b9b2-89aa-4670-87fc-a93d899b33fc
Visiting: Volkswagen Caddy at https://www.autoscout24.com/offers/volkswagen-caddy-nfz-kasten-bmt-erdgas-cng-1hand-cng-black-d31924e8-5727-4601-8b75-8d7249f49252
Visiting: Kia Ceed SW / cee'd SW at https://www.autoscout24.com/offers/kia-ceed-sw-cee-d-sw-2-hand-lpg-white-1fcd9ed1-34c3-4607-9051-d381765115fa
Visiting: Opel Zafira at https://www.autoscout24.com/offers/opel-zafira-1-6-turbo-cng-family-navi-kamera-dab-cng-black-6fc986e3-fcd1-4360-a904-ce1bb3e27052
Visiting: BMW 525 at https://www.autoscout24.

Visiting: Others  at https://www.autoscout24.com/offers/others-others-fahrzeugtransporter-haenger-brian-james-trailers-others-d0e71117-33e4-4079-88a2-4a5c5b24d23b
No 'See more' button, continuing...
Visiting: Chevrolet Captiva at https://www.autoscout24.com/offers/chevrolet-captiva-2-4-lt-gasanlage-navi-7-sitze-tuev-neu-lpg-white-42256208-14b2-4e30-9ee1-c8b313aae6ca
Visiting: Dacia Sandero at https://www.autoscout24.com/offers/dacia-sandero-stepway-extreme-1-0-tce-lpg-eco-g-kame-lpg-green-629bdd21-5d23-4b6e-96dc-c7e87a3f5f21
Visiting: Ford Mustang at https://www.autoscout24.com/offers/ford-mustang-mustang-cabrio-5-0-ti-vct-v8-aut-sehr-gepflegt-lpg-red-1496752b-225f-41a8-bb9c-fdf4cacaf1ce
Error on listing, skipping: Message: Unable to locate element: div.TieredPricingRatingsSection_nameContainer__fMSj2; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/c

Visiting: SEAT Ibiza at https://www.autoscout24.com/offers/seat-ibiza-style-1-0tgi-cng-white-776aaaa5-7509-45f3-9f51-e3bc08f9e6b3
Visiting: Volkswagen Golf at https://www.autoscout24.com/offers/volkswagen-golf-vii-1-5-tgi-dsg-iq-drive-navi-led-acc-others-white-e572dbf6-6350-4731-bd27-cd5c590f8e11
Visiting: Dacia Logan at https://www.autoscout24.com/offers/dacia-logan-mcv-ii-kombi-laureate-lpg-lpg-blue-fda86e79-3374-42fa-ae14-d45e00fe21d1
Visiting: Audi Q2 at https://www.autoscout24.com/offers/audi-q2-q2-1-0-30-tfsi-17zoll-kamera-led-navi-shz-gra-others-white-7aa8702d-a191-4689-971f-c91b7442726f
Visiting: Dodge Durango at https://www.autoscout24.com/offers/dodge-durango-5-7l-r-t-hemi-orange-srt-lpg-acc-lpg-white-57015ef6-b11a-4cbb-bfc0-e763c2ba7e23
Visiting: Dacia Sandero at https://www.autoscout24.com/offers/dacia-sandero-stepway-lpg-grey-3ec5c21f-8766-490d-8ba8-ff6c5563b7e6
Visiting: Others  at https://www.autoscout24.com/offers/others-others-p70-others-green-3652fc06-db61-480f-8740-5

### 3. Creamos un bucle para abrir cada anuncio de venta de coches en una pestaña nueva, recopilar la información contenida y finalmente cerrarlo para acto seguido repetir el proceso con el siguiente

In [ ]:
# 2) Define a helper that splits a multi-line column into numbered columns
def split_multivalue_column(df, col_name, prefix):
    # Split on newline, expand into separate columns
    splits = df[col_name].str.split('\n', expand=True)
    # Rename columns: prefix_1, prefix_2, …
    splits.columns = [f"{prefix}_{i+1}" for i in splits.columns]
    # Drop the original and join the new columns
    return df.drop(columns=[col_name]).join(splits)

# 3) Apply it to each of your four equipment columns
df = split_multivalue_column(df, 'Comfort & Convenience', 'comfort_convenience')
df = split_multivalue_column(df, 'Entertainment & Media', 'entertainment_media')
df = split_multivalue_column(df, 'Safety & Security', 'safety_security')
df = split_multivalue_column(df, 'Extras', 'extras')

# 4) Inspect the result
print(df.head())

In [ ]:
# Define the groups you want to expand
groups = {
    'comfort_convenience': [f'comfort_convenience_{i}' for i in range(1, 31)],
    # … etc …
}

# Build dummies exactly as before
dummy_frames = []
for prefix, cols in groups.items():
    existing = [c for c in cols if c in df.columns]
    stacked = df[existing].astype(str).stack()
    dummies = pd.get_dummies(stacked, prefix=prefix).groupby(level=0).max()
    dummy_frames.append(dummies)

df_expanded = pd.concat([df] + dummy_frames, axis=1)
print(df_expanded.head())

### 4. Guardamos como CSV

In [11]:
import csv

print("Introduzca la ruta en la que quiere guardar el dataframe")
ruta_csv = input("Ruta: ")
# C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2
# coches_db36_german_used_other-fuels
# db_merged_expanded_2
print()
print("Ahora escoja un nombre para el archivo")
nombre_archivo_csv = input("Nombre del archivo: ")
print()

df.to_csv(ruta_csv + "/" + nombre_archivo_csv + ".csv", sep = ';')
print("Se grabó la información correctamente")

Introduzca la ruta en la que quiere guardar el dataframe
Ruta: C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2

Ahora escoja un nombre para el archivo
Nombre del archivo: db_merged_expanded_2

Se grabó la información correctamente


### 5. Lo cargamos desde el CSV para comprobar que funciona

In [ ]:
df = pd.read_csv(ruta_csv + "/" + nombre_archivo_csv + ".csv", encoding = "utf-8",
     sep = ";", engine = "python", index_col = 0)
df

### 6. Lo guardamos en formato excel

In [12]:
print("Introduzca la ruta en la que quiere guardar el dataframe")
ruta_excel = input("Ruta: ")
# C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2
# coches_db1_german_new
print()
print("Ahora escoja un nombre para el archivo")
nombre_archivo_excel = input("Nombre del archivo: ")
print()
escritor = pd.ExcelWriter(ruta_excel + "/" + nombre_archivo_excel + ".xlsx", engine = 'xlsxwriter')
df.to_excel(escritor, sheet_name="hoja1", index=True)
escritor.save()
print("Se grabó la información correctamente")

Introduzca la ruta en la que quiere guardar el dataframe
Ruta: C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2

Ahora escoja un nombre para el archivo
Nombre del archivo: db_merged_expanded_2

Se grabó la información correctamente


### 7. Lo volvemos a cargar para comprobar que funciona

In [ ]:
df = pd.read_excel(ruta_excel + "/" + nombre_archivo_excel + ".xlsx", index_col = 0)
print(df)

# Juntamos los dataframes

In [1]:
import pandas as pd

df1 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db1_german_new.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df1 = df1.drop(0)
df1  ['_src'] = 1
df2 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db2_german_used.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df2 = df2.drop(0)
df2  ['_src'] = 2
df3 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db3_german_new_electric.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df3 = df3.drop(0)
df3  ['_src'] = 3
df4 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db4_german_used_electric.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df4 = df4.drop(0)
df4  ['_src'] = 4
df5 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db5_german_new_hybrid_electric-gasoline.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df5 = df5.drop(0)
df5  ['_src'] = 5
df6 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db6_german_used_hybrid_electric-gasoline.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df6 = df6.drop(0)
df6  ['_src'] = 6
df7 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db7_german_new_hybrid_electric-diesel.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df7 = df7.drop(0)
df7  ['_src'] = 7
df8 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db8_german_used_hybrid_electric-diesel.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df8 = df8.drop(0)
df8  ['_src'] = 8
df9 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db9_german_new_7000-20000.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df9 = df9.drop(0)
df9  ['_src'] = 9
df10 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db10_german_used_7000-20000.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df10 = df10.drop(0)
df10  ['_src'] = 10
df11 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db11_german_new_electric_hybridgasoline-electric_15000-25000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df11 = df11.drop(0)
df11  ['_src'] = 11
df12 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db12_german_used_electric_hybridgasoline-electric_15000-25000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df12 = df12.drop(0)
df12  ['_src'] = 12
df13 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db13_german_new_electric_hybridgasoline-electric_15000-25000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df13 = df13.drop(0)
df13  ['_src'] = 13
df14 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db14_german_used_electric_hybridgasoline-electric_15000-25000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df14 = df14.drop(0)
df14  ['_src'] = 14
df15 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db15_german_new_gasoline_diesel_15000-25000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df15 = df15.drop(0)
df15  ['_src'] = 15
df16 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db16_german_used_gasoline_diesel_15000-25000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df16 = df16.drop(0)
df16  ['_src'] = 16
df17 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db17_german_new_gasoline_diesel_15000-25000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df17 = df17.drop(0)
df17  ['_src'] = 17
df18 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db18_german_used_gasoline_diesel_15000-25000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df18 = df18.drop(0)
df18  ['_src'] = 18
df19 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db19_german_new_electric_hybridgasoline-electric_25000-40000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df19 = df19.drop(0)
df19  ['_src'] = 19
df20 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db20_german_used_electric_hybridgasoline-electric_25000-40000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df20 = df20.drop(0)
df20  ['_src'] = 20
df21 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db21_german_new_electric_hybridgasoline-electric_25000-40000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df21 = df21.drop(0)
df21  ['_src'] = 21
df22 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db22_german_used_electric_hybridgasoline-electric_25000-40000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df22 = df22.drop(0)
df22  ['_src'] = 22
df23 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db23_german_new_gasoline_diesel_25000-40000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df23 = df23.drop(0)
df23  ['_src'] = 23
df24 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db24_german_used_gasoline_diesel_25000-40000_power-ascending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df24 = df24.drop(0)
df24  ['_src'] = 24
df25 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db25_german_new_gasoline_diesel_25000-40000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df25 = df25.drop(0)
df25  ['_src'] = 25
df26 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db26_german_used_gasoline_diesel_25000-40000_power-descending.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df26 = df26.drop(0)
df26  ['_src'] = 26
df27 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db27_german_new_electric_hybridgasoline-electric_40000-75000.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df27 = df27.drop(0)
df27  ['_src'] = 27
df28 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db28_german_used_electric_hybridgasoline-electric_40000-75000.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df28 = df28.drop(0)
df28  ['_src'] = 28
df29 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db29_german_new_gasoline_diesel_40000-75000.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df29 = df29.drop(0)
df29  ['_src'] = 29
df30 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db30_german_used_gasoline_diesel_40000-75000.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df30 = df30.drop(0)
df30  ['_src'] = 30
df31 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db31_german_new_electric_hybridgasoline-electric_75000+.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df31 = df31.drop(0)
df31  ['_src'] = 31
df32 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db32_german_used_electric_hybridgasoline-electric_75000+.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df32 = df32.drop(0)
df32  ['_src'] = 32
df33 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db33_german_new_gasoline_diesel_75000+.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df33 = df33.drop(0)
df33  ['_src'] = 33
df34 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db34_german_used_gasoline_diesel_75000+.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df34 = df34.drop(0)
df34  ['_src'] = 34
df35 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db35_german_new_other-fuels.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df35 = df35.drop(0)
df35  ['_src'] = 35
df36 = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/coches_db36_german_used_other-fuels.csv",
                 encoding = "utf-8", sep = ";", engine = "python", index_col = 0)
df36 = df36.drop(0)
df36  ['_src'] = 36

In [2]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16,
               df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31,
               df32, df33, df34, df35, df36], ignore_index=True)
df = df.drop("Unnamed: 4", axis = 1)

In [3]:
# (3) Assign “Fuel type” based on the source tag, for every row:
df.loc[df['_src'].isin([3,  4 ]), 'Fuel type'] = 'Electric'
df.loc[df['_src'].isin([5,  6 ]), 'Fuel type'] = 'Electric/Gasoline'
df.loc[df['_src'].isin([7,  8 ]), 'Fuel type'] = 'Electric/Diesel'

# 1) List all columns except “Fuel type” and “_src”
cols_to_check = [c for c in df.columns if c not in ['Fuel type', '_src']]

# 2) Drop duplicates based only on those columns
df = df.drop_duplicates(subset=cols_to_check, keep='first').reset_index(drop=True)

In [4]:
display(df)

,car_make,car_model,price,Body type,Type,Drivetrain,Seats,Doors,Country version,Offer number,...,Load width,Load length,Payload,Perm. GVW,Wheels distance,Taxi or rental car,Electric Range (EAER)\n7,Axle count,Admissible haulage weight,Unnamed: 26
0,Dacia,Bigster,"€ 34,906",Off-Road/Pick-up,New,4WD,5.0,5.0,Germany,NW-SB-A2416-JS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nissan,NaN,"€ 21,900",Off-Road/Pick-up,New,4WD,7.0,5.0,NaN,Z0005818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Renault,Koleos,"€ 19,900",Off-Road/Pick-up,New,4WD,5.0,5.0,NaN,SC409002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Porsche,911,"€ 146,870",Coupe,New,Rear,4.0,2.0,Germany,N5236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lamborghini,Huracán,"€ 349,850",Coupe,New,4WD,2.0,2.0,Germany,M-35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,Others,NaN,"€ 649,000",Other,Used,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10893,Skoda,Praktik,"€ 1,699",Transporter,Used,Front,2.0,3.0,Germany,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10894,Volkswagen,Caddy,"€ 9,950",Transporter,Used,Front,2.0,1.0,European Union,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10895,Dacia,Dokker,"€ 5,500",Transporter,Used,Front,2.0,4.0,Germany,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df["Fuel type"].unique()

array(['Gasoline', 'Diesel', 'Super 95', 'Electric/Gasoline', nan, 'LPG',
       'Electric/Diesel', 'Super E10 95', 'Regular/Benzine 91',
       'Diesel (Particle filter) / Biodiesel / Vegetable oil', 'CNG',
       'Electric', 'Regular/Benzine 91 (Particle filter)',
       'Super E10 95 (Particle filter)', 'Super 95 (Particle filter)',
       'Diesel (Particle filter)', 'Super Plus 98', 'Super Plus E10 98',
       'Regular/Benzine E10 91', 'Super Plus E10 98 (Particle filter)',
       'Others', 'Others (Particle filter)',
       'Domestic gas H / Super 95 / Super Plus 98 / Super E10 95',
       'Domestic gas H (Particle filter) / Super 95 / Super Plus 98',
       'Liquid petroleum gas (LPG) / Super 95',
       'Domestic gas L / Super 95', 'Biogas / Super 95 / Domestic gas H',
       'Domestic gas H / Super E10 95 / Regular/Benzine 91 / Super Plus E10 98 / Super 95 / Super Plus 98 / Domestic gas L / Regular/Benzine E10 91',
       'Liquid petroleum gas (LPG) / Regular/Benzine 91 / Super

In [29]:
df.columns

Index(['car_make', 'car_model', 'price', 'Body type', 'Type', 'Drivetrain',
       'Seats', 'Doors', 'Country version', 'Offer number', 'Model code',
       'Warranty', 'Mileage', 'Production date', 'Power', 'Gearbox',
       'Engine size', 'Gears', 'Cylinders', 'Empty weight', 'Emission class',
       'Emissions sticker', 'Fuel type', 'CO₂-emissions', 'Colour',
       'Manufacturer colour', 'Paint', 'Upholstery colour', 'Upholstery',
       'dealer', 'ratings_number', 'car_location', 'General inspection',
       'Full service history', 'Non-smoker vehicle', 'Comfort & Convenience',
       'Entertainment & Media', 'Safety & Security', 'Extras',
       'Other fuel types', 'Fuel consumption', 'Energy efficiency class',
       'CO₂-efficiency', 'Availability', 'Electric Range (EAER)',
       'Available from', 'Electric Range', 'Battery Ownership',
       'Charging time from 10% to 80%', 'First registration',
       'Battery certificate', 'Cash price', 'Down payment', 'Term',
       'Net l

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10897 entries, 0 to 10896
Data columns (total 80 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   car_make                       10897 non-null  object 
 1   car_model                      10499 non-null  object 
 2   price                          10897 non-null  object 
 3   Body type                      10897 non-null  object 
 4   Type                           10897 non-null  object 
 5   Drivetrain                     8945 non-null   object 
 6   Seats                          10298 non-null  float64
 7   Doors                          10562 non-null  float64
 8   Country version                8742 non-null   object 
 9   Offer number                   9391 non-null   object 
 10  Model code                     6941 non-null   object 
 11  Warranty                       5750 non-null   object 
 12  Mileage                        10346 non-null 

In [5]:
for c in df.columns:
    if df[c].notna().sum() < 90:
        df = df.drop(c, axis = 1)          

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10897 entries, 0 to 10896
Data columns (total 67 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   car_make                       10897 non-null  object 
 1   car_model                      10499 non-null  object 
 2   price                          10897 non-null  object 
 3   Body type                      10897 non-null  object 
 4   Type                           10897 non-null  object 
 5   Drivetrain                     8945 non-null   object 
 6   Seats                          10298 non-null  float64
 7   Doors                          10562 non-null  float64
 8   Country version                8742 non-null   object 
 9   Offer number                   9391 non-null   object 
 10  Model code                     6941 non-null   object 
 11  Warranty                       5750 non-null   object 
 12  Mileage                        10346 non-null 

In [6]:
# 2) Define a helper that splits a multi-line column into numbered columns
def split_multivalue_column(df, col_name, prefix):
    # Split on newline, expand into separate columns
    splits = df[col_name].str.split('\n', expand=True)
    # Rename columns: prefix_1, prefix_2, …
    splits.columns = [f"{prefix}_{i+1}" for i in splits.columns]
    # Drop the original and join the new columns
    return df.drop(columns=[col_name]).join(splits)

# 3) Apply it to each of your four equipment columns
df = split_multivalue_column(df, 'Comfort & Convenience', 'comfort_convenience')
df = split_multivalue_column(df, 'Entertainment & Media', 'entertainment_media')
df = split_multivalue_column(df, 'Safety & Security', 'safety_security')
df = split_multivalue_column(df, 'Extras', 'extras')

# 4) Inspect the result
print(df.head())

      car_make car_model      price         Body type Type Drivetrain  Seats  \
0        Dacia   Bigster   € 34,906  Off-Road/Pick-up  New        4WD    5.0   
1       Nissan       NaN   € 21,900  Off-Road/Pick-up  New        4WD    7.0   
2      Renault    Koleos   € 19,900  Off-Road/Pick-up  New        4WD    5.0   
3      Porsche       911  € 146,870             Coupe  New       Rear    4.0   
4  Lamborghini   Huracán  € 349,850             Coupe  New        4WD    2.0   

   Doors Country version    Offer number  ... extras_13 extras_14 extras_15  \
0    5.0         Germany  NW-SB-A2416-JS  ...       NaN       NaN       NaN   
1    5.0             NaN        Z0005818  ...       NaN       NaN       NaN   
2    5.0             NaN        SC409002  ...       NaN       NaN       NaN   
3    2.0         Germany           N5236  ...       NaN       NaN       NaN   
4    2.0         Germany            M-35  ...      None      None      None   

   extras_16 extras_17 extras_18 extras_19  

In [25]:
# Valores nulos
for c in df.columns:
    print("Missing values [{0}]:".format(c), df[c].isna().sum())
print()

# Valores únicos    
for c in df.columns:
    print("Unique values [{0}]:".format(c), df[c].unique().size)

Missing values [car_make]: 0
Missing values [car_model]: 398
Missing values [price]: 0
Missing values [Body type]: 0
Missing values [Type]: 0
Missing values [Drivetrain]: 1952
Missing values [Seats]: 599
Missing values [Doors]: 335
Missing values [Country version]: 2155
Missing values [Offer number]: 1506
Missing values [Model code]: 3956
Missing values [Warranty]: 5147
Missing values [Mileage]: 551
Missing values [Production date]: 8809
Missing values [Power]: 3
Missing values [Gearbox]: 77
Missing values [Engine size]: 2269
Missing values [Gears]: 5565
Missing values [Cylinders]: 4514
Missing values [Empty weight]: 4057
Missing values [Emission class]: 3465
Missing values [Emissions sticker]: 1427
Missing values [Fuel type]: 2118
Missing values [CO₂-emissions]: 1726
Missing values [Colour]: 498
Missing values [Manufacturer colour]: 791
Missing values [Paint]: 1871
Missing values [Upholstery colour]: 1384
Missing values [Upholstery]: 569
Missing values [dealer]: 0
Missing values [rati

Unique values [extras_3]: 34
Unique values [extras_4]: 32
Unique values [extras_5]: 31
Unique values [extras_6]: 31
Unique values [extras_7]: 26
Unique values [extras_8]: 24
Unique values [extras_9]: 22
Unique values [extras_10]: 22
Unique values [extras_11]: 20
Unique values [extras_12]: 19
Unique values [extras_13]: 15
Unique values [extras_14]: 14
Unique values [extras_15]: 13
Unique values [extras_16]: 12
Unique values [extras_17]: 10
Unique values [extras_18]: 9
Unique values [extras_19]: 7
Unique values [extras_20]: 6
Unique values [extras_21]: 5
Unique values [extras_22]: 3


In [1]:
# comfort_convenience_38
# entertainment_media_16
# safety_security_34
# extras_22
import pandas as pd

df = pd.read_csv("C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2/db_merged.csv",
                encoding = "utf-8", sep = ";", engine = "python", index_col = 0)

In [7]:
# Step 1: Collect all the comfort_convenience_* column names
comfort_cols = [c for c in df.columns if c.startswith('comfort_convenience_')]
entertainment_cols = [c for c in df.columns if c.startswith('entertainment_media_')]
safety_cols = [c for c in df.columns if c.startswith('safety_security_')]
extras_cols = [c for c in df.columns if c.startswith('extras_')]

# Step 2: For each row, count how many of those columns are non-null (i.e. actually have a feature)
df['comfort_convenience_elements'] = df[comfort_cols].notna().sum(axis=1)
df['entertainment_media_elements'] = df[entertainment_cols].notna().sum(axis=1)
df['safety_security_elements'] = df[safety_cols].notna().sum(axis=1)
df['extras_elements'] = df[extras_cols].notna().sum(axis=1)

In [8]:
df

,car_make,car_model,price,Body type,Type,Drivetrain,Seats,Doors,Country version,Offer number,...,extras_17,extras_18,extras_19,extras_20,extras_21,extras_22,comfort_convenience_elements,entertainment_media_elements,safety_security_elements,extras_elements
0,Dacia,Bigster,"€ 34,906",Off-Road/Pick-up,New,4WD,5.0,5.0,Germany,NW-SB-A2416-JS,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,Nissan,NaN,"€ 21,900",Off-Road/Pick-up,New,4WD,7.0,5.0,NaN,Z0005818,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,Renault,Koleos,"€ 19,900",Off-Road/Pick-up,New,4WD,5.0,5.0,NaN,SC409002,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,Porsche,911,"€ 146,870",Coupe,New,Rear,4.0,2.0,Germany,N5236,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Lamborghini,Huracán,"€ 349,850",Coupe,New,4WD,2.0,2.0,Germany,M-35,...,NaN,NaN,NaN,NaN,NaN,NaN,20,10,18,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,Others,NaN,"€ 649,000",Other,Used,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
10893,Skoda,Praktik,"€ 1,699",Transporter,Used,Front,2.0,3.0,Germany,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,2,8,3
10894,Volkswagen,Caddy,"€ 9,950",Transporter,Used,Front,2.0,1.0,European Union,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
10895,Dacia,Dokker,"€ 5,500",Transporter,Used,Front,2.0,4.0,Germany,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,4,8,0


In [9]:
# Define the groups you want to expand
groups = {
    'comfort_convenience': [f'comfort_convenience_{i}' for i in range(1, 39)],
    'entertainment_media': [f'entertainment_media_{i}' for i in range(1, 17)],
    'safety_security': [f'safety_security_{i}' for i in range(1, 35)],
    'extras': [f'extras_{i}' for i in range(1, 23)]
    # … etc …
}

# Build dummies exactly as before
dummy_frames = []
for prefix, cols in groups.items():
    existing = [c for c in cols if c in df.columns]
    stacked = df[existing].astype(str).stack()
    dummies = pd.get_dummies(stacked, prefix=prefix).groupby(level=0).max()
    dummy_frames.append(dummies)

df_expanded = pd.concat([df] + dummy_frames, axis=1)
print(df_expanded.head())

      car_make car_model      price         Body type Type Drivetrain  Seats  \
0        Dacia   Bigster   € 34,906  Off-Road/Pick-up  New        4WD    5.0   
1       Nissan       NaN   € 21,900  Off-Road/Pick-up  New        4WD    7.0   
2      Renault    Koleos   € 19,900  Off-Road/Pick-up  New        4WD    5.0   
3      Porsche       911  € 146,870             Coupe  New       Rear    4.0   
4  Lamborghini   Huracán  € 349,850             Coupe  New        4WD    2.0   

   Doors Country version    Offer number  ... extras_Sport suspension  \
0    5.0         Germany  NW-SB-A2416-JS  ...                       0   
1    5.0             NaN        Z0005818  ...                       0   
2    5.0             NaN        SC409002  ...                       0   
3    2.0         Germany           N5236  ...                       0   
4    2.0         Germany            M-35  ...                       1   

  extras_Steel wheels extras_Summer tyres  extras_Touch screen  \
0             

In [10]:
df_expanded

,car_make,car_model,price,Body type,Type,Drivetrain,Seats,Doors,Country version,Offer number,...,extras_Sport suspension,extras_Steel wheels,extras_Summer tyres,extras_Touch screen,extras_Trailer hitch,extras_Tuned car,extras_Voice Control,extras_Winter package,extras_Winter tyres,extras_nan
0,Dacia,Bigster,"€ 34,906",Off-Road/Pick-up,New,4WD,5.0,5.0,Germany,NW-SB-A2416-JS,...,0,0,0,0,0,0,0,0,0,1
1,Nissan,NaN,"€ 21,900",Off-Road/Pick-up,New,4WD,7.0,5.0,NaN,Z0005818,...,0,0,0,0,0,0,0,0,0,1
2,Renault,Koleos,"€ 19,900",Off-Road/Pick-up,New,4WD,5.0,5.0,NaN,SC409002,...,0,0,0,0,0,0,0,0,0,1
3,Porsche,911,"€ 146,870",Coupe,New,Rear,4.0,2.0,Germany,N5236,...,0,0,0,0,0,0,0,0,0,1
4,Lamborghini,Huracán,"€ 349,850",Coupe,New,4WD,2.0,2.0,Germany,M-35,...,1,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,Others,NaN,"€ 649,000",Other,Used,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
10893,Skoda,Praktik,"€ 1,699",Transporter,Used,Front,2.0,3.0,Germany,NaN,...,0,1,0,0,0,0,0,0,0,0
10894,Volkswagen,Caddy,"€ 9,950",Transporter,Used,Front,2.0,1.0,European Union,NaN,...,0,0,0,0,0,0,0,0,0,1
10895,Dacia,Dokker,"€ 5,500",Transporter,Used,Front,2.0,4.0,Germany,NaN,...,0,0,0,0,0,0,0,0,0,1
